# 2.5D DC Resistivity Forward Simulation

## Goals

- Create a synthetic DC survey
- Define a mesh we'll use to simulate the forward problem
- Run a forward problem for a 2.5D resistivity model

<img src="https://raw.githubusercontent.com/simpeg/agrogeo24/main/images/dcr_fwd.png" width="100%" align="center"/>

## Step 0: Importing Modules

In [ ]:
# SimPEG functionality
from SimPEG.electromagnetics.static import resistivity as dc
from SimPEG.utils import model_builder
from SimPEG import maps
from SimPEG.electromagnetics.static.utils.static_utils import (
    generate_dcip_sources_line,
    pseudo_locations,
    plot_pseudosection,
    apparent_resistivity_from_voltage,
)

# discretize functionality
from discretize import TreeMesh
from discretize.utils import active_from_xyz

# Common Python functionality
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

mpl.rcParams.update({"font.size": 14})  # default font size

write_output = True  # Optional

## Step 1: Defining Topography

Lets define a topography:

- Spans over -300m to 300m
- Has a horizontal resolution of 1m
- The synthetic elevations are generated with a function:
    $$ y(x) = 4 + 4 \tanh \left( \frac{x}{100} \right) $$

In [ ]:
fig = plt.figure(figsize=(6, 2))
ax = fig.add_axes([0.1, 0.1, 0.8, 0.8])
ax.plot(topo_2d[:, 0], topo_2d[:, -1], color="b", linewidth=2)
ax.set_xlabel("x (m)", labelpad=5)
ax.set_ylabel("y (m)", labelpad=5)
ax.grid(True)
ax.set_title("Topography", fontsize=16, pad=10)
plt.show(fig)

## Step 2: Defining the Survey

<img src="https://raw.githubusercontent.com/simpeg/agrogeo24/main/images/DCR_DipoleDipole_Array.svg" width="40%" align="right"/>

DC resistivity surveys within SimPEG require the user to create and connect three types of objects:

- receivers
- sources
- survey

These can be defined manually, or efficiently using SimPEG utilities like `generate_dcip_sources_line()`

In [ ]:
# Plot pseudo locations
pseudo_locations_xy = pseudo_locations(survey)

fig = plt.figure(figsize=(8, 2.75))
ax = fig.add_axes([0.1, 0.1, 0.85, 0.8])
ax.scatter(pseudo_locations_xy[:, 0], pseudo_locations_xy[:, -1], 8, "r")
ax.set_xlabel("x (m)")
ax.set_ylabel("y (m)")
ax.set_title("Pseudo-locations")
plt.show()

## Step 3: Designing a Mesh

Meshes define the numerical grid on which we numerically solve the PDE for the DC resistivity problem.

<img src="https://raw.githubusercontent.com/simpeg/agrogeo24/main/images/sphx_glr_1_mesh_overview_001.png" width="75%" align="center"/>

Here, we define a tree mesh with the following properties:

- Minimum cell size of 0.5m
- Spans 800m in the horizontal direction
- Spans 400m in the vertical direction 

In [ ]:
# Number of base cells along x and y
nbcx = 2 ** int(np.round(np.log(dom_width_x / dh) / np.log(2.0)))
nbcy = 2 ** int(np.round(np.log(dom_width_y / dh) / np.log(2.0)))

# Define the base mesh with top at z = 0 m.
hx = [(dh, nbcx)]
hy = [(dh, nbcy)]
mesh = TreeMesh([hx, hy], origin="CN")

# Shift top to maximum topography and center of survey line
y_topo_max = np.max(topo_2d[:, -1])
mesh.origin = mesh.origin + np.r_[np.mean(end_locations), y_topo_max]

In [ ]:
# Mesh refinement near electrodes.
unique_locations = survey.unique_electrode_locations

mesh.refine_points(
    unique_locations, padding_cells_by_level=[10, 8, 8, 8, 4, 4], finalize=False
)

# Mesh refinement based on topography
mesh.refine_surface(
    topo_2d[np.abs(x_topo) < 150.0, :],
    padding_cells_by_level=[0, 0, 4, 4],
    finalize=False,
)

In [ ]:
# Finalize
mesh.finalize()

In [ ]:
fig = plt.figure(figsize=(8, 4))

ax1 = fig.add_axes([0.14, 0.17, 0.8, 0.7])
mesh.plot_grid(ax=ax1, linewidth=1)
ax1.grid(False)
ax1.set_xlim(-200, 200)
ax1.set_ylim(y_topo_max - 200, y_topo_max)
ax1.set_title("Mesh")
ax1.set_xlabel("x (m)")
ax1.set_ylabel("y (m)")

plt.show()

Once the tree mesh is initialized, we want to refine it close to the electrodes and around the topography

In [ ]:
# Plot the mesh
fig = plt.figure(figsize=(8, 4))

ax1 = fig.add_axes([0.14, 0.17, 0.8, 0.7])
mesh.plot_grid(ax=ax1, linewidth=1)
ax1.grid(False)
ax1.set_xlim(-200, 200)
ax1.set_ylim(np.max(y_topo) - 200, np.max(y_topo))
ax1.set_title("Mesh")
ax1.set_xlabel("x (m)")
ax1.set_ylabel("y (m)")

plt.show()

We can get properties of the mesh (number of cells, cell centers, etc):

## Step 4: Active Cells

We can fix the resistivity of some cells (inactive cells), so our model only modifies the resistivity of the **active cells**.

We can use `active_from_xyz` to define active cells from the topography:
- Cells above the topography will be **inactive**.
- Cells below the topography will be **active**.

<img src="https://raw.githubusercontent.com/simpeg/agrogeo24/main/images/active_cells.png" width="20%"/>

## Step 5: Defining a Model

Lets generate a model with the resistivities of each active cell:

* Near surface resistivity of 100 $\Omega m$
* Basement resistivity of 10 $\Omega m$ ( depths < -16m )
* An infinite block (TODO: ADD DIMENSIONS OF THE BLOCK)

Lets start by defining a model with only the background resistivity

Add the basement

Add the block with [`model_builder.get_indices_block`](http://docs.simpeg.xyz/content/api/generated/SimPEG.utils.model_builder.get_indices_block.html).

Keep only the elements that correspond to the active cells

In [ ]:
plotting_map = maps.InjectActiveCells(mesh, active_cells, np.nan)

fig = plt.figure(figsize=(10, 3))

norm = LogNorm(vmin=1e1, vmax=1e3)

ax1 = fig.add_axes([0.14, 0.17, 0.68, 0.7])
mesh.plot_image(
    plotting_map * resistivity_model,
    ax=ax1,
    grid=False,
    pcolor_opts={"norm": norm, "cmap": mpl.cm.RdYlBu_r},
)
ax1.set_xlim(-70, 70)
ax1.set_ylim(y_topo_max - 50, y_topo_max)
ax1.set_title("Electrical Resistivity (Active Cells)")
ax1.set_xlabel("x (m)")
ax1.set_ylabel("y (m)")

ax2 = fig.add_axes([0.84, 0.17, 0.03, 0.7])
cbar = mpl.colorbar.ColorbarBase(
    ax2, norm=norm, orientation="vertical", cmap=mpl.cm.RdYlBu_r
)
cbar.set_label(r"$\rho (\Omega m)$", rotation=270, labelpad=25, size=16)

plt.show()

## Step 6: Mapping from the Model to the Mesh

Define a model representing electrical resistivities on all active cells. Use the [`maps.InjectActiveCells`](myst:SimPEG#SimPEG.maps.InjectActiveCells) mapping to define this mapping. 

> **Important:**
> Although the true electrical resistivity of the air is infinity, set the value for the inactive cells to 1e8 $\Omega m$!

<img src="https://raw.githubusercontent.com/simpeg/agrogeo24/main/images/mapping_1.png" width="50%" align="center"/>

## Step 7: Project Survey to Discretized Topography

We should shift the vertical coordinate of the dipoles so they are lying on top of the discrete surface.

Lets use [`Survey.drape_electrodes_on_topography`](https://docs.simpeg.xyz/content/api/generated/SimPEG.electromagnetics.static.resistivity.Survey.drape_electrodes_on_topography.html#SimPEG.electromagnetics.static.resistivity.Survey.drape_electrodes_on_topography) for that.

<img src="https://raw.githubusercontent.com/simpeg/agrogeo24/main/images/project_electrodes.png" width="40%" align="center"/>

## Step 8: Defining the Forward Simulation

Lets create a simulation: the object that will simulate the physics!

We'll use [dc.simulation_2d.Simulation2DNodel](https://docs.simpeg.xyz/content/api/generated/SimPEG.electromagnetics.static.resistivity.Simulation2DNodal.html#SimPEG.electromagnetics.static.resistivity.Simulation2DNodal). 

<img src="https://raw.githubusercontent.com/simpeg/agrogeo24/main/images/fwd_simulation.png" width="30%"/>

## Step 9: Predict DC Resistivity Data and Plot

We can use the **dpred** method to simulate DC resistivity data for your resistivity model.

## Plot DC Resistivity Data

Here we use the [plot_pseudosection](https://docs.simpeg.xyz/content/api/generated/SimPEG.electromagnetics.static.utils.plot_pseudosection.html#SimPEG.electromagnetics.static.utils.plot_pseudosection) utility function to represent the predicted data on a pseudosection plot as apparent conductivities. If the receivers were defined to simulate data as normalized voltages, we may want to use the [apparent_resistivity_from_voltage](https://docs.simpeg.xyz/content/api/generated/SimPEG.electromagnetics.static.utils.apparent_resistivity_from_voltage.html#SimPEG.electromagnetics.static.utils.apparent_resistivity_from_voltage) utility function to convert the data to apparent resistivities.

In [ ]:
# Plot voltages pseudo-section
if data_type == "volt":
    fig = plt.figure(figsize=(8, 2.75))
    ax1 = fig.add_axes([0.1, 0.15, 0.75, 0.78])
    plot_pseudosection(
        survey,
        dobs=np.abs(dpred),
        plot_type="scatter",
        ax=ax1,
        scale="log",
        cbar_label="V/A",
        scatter_opts={"cmap": mpl.cm.viridis},
    )
    ax1.set_title("Normalized Voltages")
    plt.show()

    # Get apparent conductivities from volts and survey geometry
    apparent_resistivity = apparent_resistivity_from_voltage(survey, dpred)

else:
    apparent_resistivity = dpred.copy()

# Plot apparent resistivity pseudo-section
fig = plt.figure(figsize=(8, 2.75))
ax1 = fig.add_axes([0.1, 0.15, 0.75, 0.78])
plot_pseudosection(
    survey,
    dobs=apparent_resistivity,
    data_locations=True,
    plot_type="contourf",
    ax=ax1,
    scale="log",
    cbar_label="$\Omega m$",
    mask_topography=True,
    contourf_opts={"levels": 40, "cmap": mpl.cm.RdYlBu_r},
)
ax1.set_title("Apparent Resistivity")
plt.show()